In [14]:
import os
import numpy as np
from PIL import Image
import random
import matplotlib as plt

In [15]:

raw_folder = "raw_images/"
annotated_folder = "annotated_images/"

occupied_dir = "training_data/occupied/"
empty_dir = "training_data/empty/"

if not os.path.exists(occupied_dir):
    os.makedirs(occupied_dir)

if not os.path.exists(empty_dir):
    os.makedirs(empty_dir)

In [16]:
empty_images = os.listdir(empty_dir)
occupied_images = os.listdir(occupied_dir)

print(f"Number of empty parking spaces detected: {len(empty_images)}")
print(f"Number of occupied parking spaces detected: {len(occupied_images)}")


Number of empty parking spaces detected: 0
Number of occupied parking spaces detected: 0


In [17]:
import pandas as pd
import os
from PIL import Image, ImageDraw
import numpy as np
import ast

# Directories
raw_folder = "raw_images/"
occupied_dir = "training_data/occupied/"
empty_dir = "training_data/empty/"

if not os.path.exists(occupied_dir):
    os.makedirs(occupied_dir)

if not os.path.exists(empty_dir):
    os.makedirs(empty_dir)

# Load CSV
df = pd.read_csv("detection-dataset.csv")

def extract_parking_spot(image_path, polygons):
    img = Image.open(image_path).convert("RGB")
    img_array = np.array(img)
    
    for polygon_info in polygons:
        label = polygon_info['label']
        points = polygon_info['points']
        
        # Draw the polygon to get a mask
        mask_img = Image.new('L', img.size, 0)
        ImageDraw.Draw(mask_img).polygon(points, outline=1, fill=1)
        mask = np.array(mask_img).astype(bool)
        
        # Extract spot using the mask
        spot_img_array = np.zeros_like(img_array)
        spot_img_array[mask] = img_array[mask]
        spot_img = Image.fromarray(spot_img_array)
        
        # Save to appropriate folder
        if label == "free_parking_space":
            save_path = os.path.join(empty_dir, f"{os.path.basename(image_path)}_free_{points[0][0]}_{points[0][1]}.jpg")
        else:  # Assuming only two labels: free_parking_space and not_free_parking_space
            save_path = os.path.join(occupied_dir, f"{os.path.basename(image_path)}_occupied_{points[0][0]}_{points[0][1]}.jpg")
        
        spot_img.save(save_path)

# Process each image
for _, row in df.iterrows():
    image_path = os.path.join(raw_folder, row['image_name'])
    annotations = ast.literal_eval(row['annotations'])
    extract_parking_spot(image_path, annotations)


In [18]:
empty_images = os.listdir(empty_dir)
occupied_images = os.listdir(occupied_dir)

print(f"Number of empty parking spaces detected: {len(empty_images)}")
print(f"Number of occupied parking spaces detected: {len(occupied_images)}")


Number of empty parking spaces detected: 273
Number of occupied parking spaces detected: 630
